# Практика
Используемые библиотеки

In [2]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [3]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [54]:
# Ваше решение
def check_value(value, condition):
    condition = condition.replace(',', '.')
    if value == "б/цвета":
        value = 0
    if 'в пределах' in condition:
        s = condition.split(' ')[2]
        values = s.split('-')
        if float(value) > float(values[0]) and float(value) < float(values[1]):
            return True
        return False
    elif 'не более' in condition:
        value1 = condition.split(' ')[2]
        if '-' in value1:
            value2 = value1.split('-')[0]
            if float(value2) > float(value):
                return True
            else:
                return False
        else: 
            if float(value1) > float(value):
                return True
            else:
                return False
    else:
        return False


In [55]:
from tqdm import tqdm
tqdm.pandas(desc='Progress:')
df["OK"] = df.progress_apply(lambda x:check_value(x["Результат анализа"], x["Норматив"]), axis='columns', result_type='expand')
df

Progress:: 100%|██████████| 14/14 [00:00<00:00, 14011.04it/s]


,Показатель,Единица измерений,Результат анализа,Норматив,OK
0,pH,единицы pH,8.4,в пределах 6-9,True
1,Запах,баллы,1,не более 2-3,True
2,Цветность,градусы,б/цвета,не более 30,True
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
6,Нитраты (по NO3),мг/дм3,24,не более 45,True
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
8,Хлориды (Cl),мг/дм3,200,не более 350,True
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [36]:
import random
random.seed(1)
# Инициализируем счетчик для выбранных фруктов
hits_counter = 0
loops = 1000000
for i in range(loops):
    # Создаем список фруктов
    fruits = ['orange'] * 5 + ['apple'] * 4
    counter_oranges = 0
    counter = 0
    while counter < 3:
        fruit = random.choice(fruits)
        # убираем из ящика выбранный фрукт для дальнейшей корректной выборки
        fruits.pop(fruits.index(fruit))
        counter += 1
        if fruit == 'orange':
            counter_oranges += 1
    if counter_oranges == 3:
        hits_counter += 1
print("The probability of getting 3 oranges is: ", hits_counter/loops)

The probability of getting 3 oranges is:  0.119051


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [40]:
# Ваше решение
import random
random.seed(1)

# Инициализируем счетчик для проверенных деталей
checked_standard_parts = 0
hits_counter = 0
loops = 1000
for i in range(loops):
    # Создаем список деталей
    parts = ['standard'] * 7 + ['non-standard'] * 3
    counter_parts = 0
    counter = 0
    while counter < 2:
        part = random.choice(parts)
        parts.pop(parts.index(part))
        if part == 'standard' and counter == 1:
            hits_counter += 1
        counter +=1 
        
print("The probability of getting 2 parts is: ", hits_counter/loops)

The probability of getting 2 parts is:  0.701
